In [ ]:
%load_ext autoreload
%matplotlib inline
%autoreload 2
%config Completer.use_jedi = True
import matplotlib as mpl
from matplotlib import pyplot as plt
mpl.rc("figure", dpi=100)
from warnings import simplefilter
# ignore all future warnings
simplefilter(action='ignore')
import scalp
from scalp.output import draw
import lmz
import numpy as np
import scanpy as sc

In [ ]:
dataset = scalp.data.mkblobs(sizes = [[15,30],[25,20],[23,22]], clusterspread = .5, batchspread = 4)
# dataset = scalp.data.mkblobs(sizes = [[10,5],[12,3],[7,8]], clusterspread = 1, batchspread = 4)
for d in dataset:
    d.obsm['umap'] = d.X
stack = scalp.transform.stack(dataset)
sc.pl.umap(stack, color=['batch', 'label'], s= 80)

In [ ]:
from scipy.sparse.csgraph import dijkstra

parm = {'neighbors_total': 10,
         'neighbors_intra_fraction': .5,
         'add_tree': False,
         'epsilon'  : 1e-6, 
          'copy_lsa_neighbors': False, 
         'inter_outlier_threshold': .95,
         'inter_outlier_probabilistic_removal': False}
dataset, graph = scalp.mkgraph(dataset,**parm)

graph = dijkstra(graph)
scalp.umapwrap.graph_xumap(dataset,graph,label = 'umap', n_components = 2)

# scalp.umapwrap.graph_umap(dataset,graph,label = 'umap', n_components = 2)

stack = scalp.transform.stack(dataset)
sc.pl.umap(stack, color=['batch', 'label'], s= 100)
# plt.matshow(graph)

import structout as so
#so.heatmap(graph)

In [ ]:

import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
from scipy.sparse import csr_matrix

def plot_sparse_matrix(sparse_matrix, plot_title="Graph"):
    """
    Converts a sparse matrix into a NetworkX graph and plots it using a spring layout.

    Parameters:
    - sparse_matrix (scipy.sparse.csr_matrix): The sparse matrix representing the graph.
    - plot_title (str): Title for the plot.

    Returns:
    - None
    """
    # Convert the sparse matrix to a NetworkX graph
    graph = nx.from_scipy_sparse_array(sparse_matrix)
    # print(graph.edges(data=True))
    # Compute the spring layout
    pos = nx.spring_layout(graph)

    # Plot the graph
    plt.figure(figsize=(8, 8))
    col = stack.obs['batch']
    nx.draw(graph, pos, with_labels=True,
            node_size=100, node_color=col,
            font_size=8, font_color='black', font_weight='bold',
            edge_color='gray', linewidths=0.5, alpha=0.7)
    plt.title(plot_title)
    plt.show
    
plot_sparse_matrix(csr_matrix(graph), plot_title="Graph")


In [ ]:
stack